In [23]:
import pandas as pd
from utils import Save_CSV, Append_Or_Create_CSV_By_Name
import os
import re

In [16]:
categories_list = []

for i in range(0,4):
    df = pd.read_csv(f"data/reviews_part_1.{i}.csv")
    for cat in df.Category.unique():
        if cat not in categories_list: 
            categories_list.append(cat)
        cat_df = df[df['Category'] == cat]
        # print(cat)
        # Append_Or_Create_CSV_By_Name(f"{cat}.csv", cat_df)

print(categories_list)

C:\Users\Omer\AppData\Local\Temp\ipykernel_16452\2452893455.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/reviews_part_1.{i}.csv")
C:\Users\Omer\AppData\Local\Temp\ipykernel_16452\2452893455.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/reviews_part_1.{i}.csv")


['דודי שמש וחשמל', 'שרותי מנעולנים', 'שרותי הובלות', 'התקנה ותיקון מזגנים']


In [18]:
for cat in categories_list:
    df = pd.read_csv(f"data/{cat}.csv")
    df.drop_duplicates(subset=['Date','Customer Name','Review', 'Professional SysID'], keep='first', inplace=True)
    Save_CSV(cat+".csv", df)

C:\Users\Omer\AppData\Local\Temp\ipykernel_16452\3192907318.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/{cat}.csv")


In [ ]:
# Merge Same Categories CSV's 

df = pd.read_csv("data\\התקנה ותיקון מזגנים.csv")
df2 = pd.read_csv("data\\AirConditioners.csv")

frames = [df, df2]

df3 = pd.concat(frames)
df3_new = df3.drop_duplicates(subset=['Date','Customer Name','Review', 'Professional SysID'],keep='first')

Save_CSV('התקנה ותיקון מזגנים.csv', df3_new)

print(f'{df3.shape[0]}, {df3.shape[0]}')


In [58]:

my_df = pd.read_csv("data/שרותי מנעולנים.csv")
my_df = df.copy()
my_df['Date'] = pd.to_datetime(my_df['Date'])
print(my_df.shape[0])

# Extract the year
my_df['year'] = my_df['Date'].dt.year
# match = re.match(r"\d+")
dd = df[df["Location"].str.contains(r'\d+',na=False, regex=True) == False]
dd["Location"].value_counts(sort=True)
# print(my_df["Location"].unique())
# print(my_df[my_df["Location"] == my_df["Customer Name"]]["year"].value_counts(sort=True))


61670


תל אביב      12654
רמת גן        3303
ירושלים       3192
פתח תקווה     2880
חיפה          2873
             ...  
כרמיה            1
חלוץ             1
גבע כרמל         1
כפר חנניה        1
גשר הזיו         1
Name: Location, Length: 682, dtype: int64

In [74]:
# Add 'Year' Column
CSV_name = "all_categories.csv"
df = pd.read_csv(f"data/{CSV_name}") 
df_to_scan = df.loc[df["Is Relevant"] == 1]
list = df_to_scan["Category Name"].unique()

for cat in list:
    df = pd.read_csv(f"data/{cat}.csv") 
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    Save_CSV(f"{cat}.csv", df)

C:\Users\Omer\AppData\Local\Temp\ipykernel_16452\3052088807.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/{cat}.csv")
